In [1]:
import numpy as np
import pandas as pd

%cd C:/Users/David/Google Drive/CSC478Proj/processing data2
#%cd C:/Users/David/Dropbox/DePaulUniv/CSC478/HW2

print
pilly_df = pd.read_csv("pilly_rows.csv", sep=',', header=0, na_values=[''], index_col=0)  #yes for headers
#yarn_all = pd.read_csv("all_rows_all_cols_reduced.csv", sep=',', header=0, na_values=[''], index_col=0)
notpilly_df = pd.read_csv("not_pilly_rows.csv", sep=',', header=0, na_values=[''], index_col=0) 
yarn_all = pd.read_csv("all_reduced_reg_norm.csv", sep=',', header=0, na_values=[''], index_col=0)

C:\Users\David\Google Drive\CSC478Proj\processing data2



In [2]:
print 'pilly ', pilly_df.shape
print 'not pilly ', notpilly_df.shape
print 'all yarn ', yarn_all.shape

yarn_all_np = np.array(yarn_all)
pilly_np = np.array(pilly_df)
notpilly_np = np.array(notpilly_df)

pilly  (153, 151)
not pilly  (77, 151)
all yarn  (3657, 151)


#### Training data distribution Pilly = 153 / (153 + 77) = 66.5%  and Not Pilly = 77 / (153 + 77)  = 33.5%

In [3]:
#print pilly_np[0:5]
#np.ones(pilly_np.shape[0]) * 2

#### Func to create new prototype Rocchio Vector with given dataset

In [4]:
def AddRocchioV(PVector, classdata, cvalue):
    #expect a numpy Prototype Vector to pass in for appending
    #a dataset with given class 'cvalue' to be computed
    #create empty class vector of ones to be populated with cvalue
    classvector = np.ones(classdata.shape[0])
    classvector =  classvector * cvalue
        
    NumRows = classdata.shape[0]
    
    for row in range(NumRows):
    #for row in range(5):         #Testing with 10 only
        PVector[cvalue] = PVector[cvalue] +  classdata[row]  
    
    #check out prototype vectors
    #for v in PVector:
    print 'Contents of Prototype Vector for Class ', cvalue
    print PVector[cvalue] [0:5]
        
    return PVector, classvector

In [5]:
#create numpy array of class labels
yarn_classes = [0, 1]
yclass_np = np.array(yarn_classes)
print yclass_np[0]
print yclass_np[1]

0
1


In [6]:
## initialize 0 vectors for Prototype Vectors with rows = Number of Classes
pV = np.zeros( (2 , yarn_all.shape[1]), dtype=np.float )
print 'shape of Prototype Vectors' , pV.shape
#print pV[0]
#print pV[1]


shape of Prototype Vectors (2L, 151L)


In [7]:
## create not pilly Prototype Vector using class 0
pV, Y_notpilly = AddRocchioV(pV, notpilly_np, 0)

Contents of Prototype Vector for Class  0
[  6.   0.  57.   0.   2.]


In [8]:
## create not pilly Prototype Vector using class 1
pV, Y_pilly = AddRocchioV(pV, pilly_np, 1)

Contents of Prototype Vector for Class  1
[  11.    3.  100.    0.    2.]


In [9]:
print Y_notpilly.shape
print Y_pilly.shape
Y_pilly[0:5]

(77L,)
(153L,)


array([ 1.,  1.,  1.,  1.,  1.])

In [10]:
def RocCoSCategorize(testdf, testclasses, protoV, uniqclasses):
    #Rocchio Text Categorization, prototype vectors for CoSine similarity required
    
    if len(protoV) <> len(uniqclasses):
        print 'Error!  # of Prototype Vectors do not match # of Unique Classes'
        
    numTestrows = len(testclasses)
    testclasses = np.array(testclasses)
    testdf      = np.array(testdf)
    errorCount = 0.0
    resultlist = []
    cosinelist=[]

    for row in range(numTestrows):
    #for row in range(5):         #Testing with 10 only
        
        currCoS = np.float(-2.0)   #init with max value for each new row
        D = testdf[row]             #doc vector for categorization
        cosine_sim = 0.000
                
        for iclass in range(len(protoV)):
            p = protoV[iclass]          #prototype vector to be compared to
            D_norm = np.linalg.norm(D)   #normalize test doc vector curr row
            p_norm = np.linalg.norm(p)
            cosine_sim = np.dot(D,p) / (D_norm * p_norm)
            #print 'curr cosine_sim for row ', row, cosine_sim
            if cosine_sim > currCoS:
                currCoS = cosine_sim
                classfound = uniqclasses[iclass]
                
        
        #store results
        resultlist.append( [classfound, testclasses[row] ]  ) 
        cosinelist.append(currCoS)
        #print 'Test Row', row, 'was classified Euclidean as %s, actual answer is %s' % (votedKNN, classTest['class'][row])
        if (classfound !=  testclasses[row]): errorCount += 1.0

    errorRate = errorCount/float(numTestrows)
    #print "the Rocchio total error rate is: %f" % (errorRate)
    return errorRate, resultlist, np.array(cosinelist)

In [11]:
## call Rocchio function 
RocchErrRate, RocPilly, PillyCoSign = RocCoSCategorize(pilly_np, Y_pilly, pV, yclass_np)
print "the Rocchio total error rate is: %f" % (RocchErrRate)
#print 'Rocchio Error results is ', Rocresults 

the Rocchio total error rate is: 0.281046


In [12]:
print PillyCoSign.shape
len(RocPilly)

(153L,)


153

In [13]:
print 'Results for Pilly'
print "the Rocchio total error rate is: %f" % (RocchErrRate)
print "the Rocchio Accuracy rate is: %f" % (1 - RocchErrRate)

Results for Pilly
the Rocchio total error rate is: 0.281046
the Rocchio Accuracy rate is: 0.718954


In [14]:
PillyCoS_df = pd.DataFrame( PillyCoSign)
PillyCoS_df.describe(include="all")

0
count  153.000000
mean     0.581518
std      0.120070
min      0.178091
25%      0.520558
50%      0.592176
75%      0.672340
max      0.794474

In [15]:
## call Rocchio function Not Pilly 
RocchErrRate, RocNotPilly, noPillyCoSign = RocCoSCategorize(notpilly_np, Y_notpilly, pV, yclass_np)
print "the Rocchio total error rate is: %f" % (RocchErrRate)
#print 'Rocchio Error results is ', Rocresults 

the Rocchio total error rate is: 0.376623


In [16]:
print 'Results for NOT Pilly'
print "the Rocchio total error rate is: %f" % (RocchErrRate)
print "the Rocchio Accuracy rate is: %f" % (1 - RocchErrRate)

Results for NOT Pilly
the Rocchio total error rate is: 0.376623
the Rocchio Accuracy rate is: 0.623377


In [18]:
noPillyCoS_df = pd.DataFrame( noPillyCoSign)
noPillyCoS_df.describe(include="all")

0
count  77.000000
mean    0.614742
std     0.116548
min     0.292673
25%     0.546897
50%     0.623740
75%     0.703718
max     0.815875

In [19]:
print 'Results for not Pilly'
print "the Rocchio total error rate is: %f" % (RocchErrRate)
print "the Rocchio Accuracy rate is: %f" % (1 - RocchErrRate)

Results for not Pilly
the Rocchio total error rate is: 0.376623
the Rocchio Accuracy rate is: 0.623377


In [17]:
print PillyCoSign[0:3]
count_PAll = PillyCoSign > 0.67 
#print count_PAll[0:5]
my_tab = pd.crosstab(index=count_PAll ,   columns="count",  margins=True  ) #  # Name the count column
print my_tab
print 'Pilly Accuracy counting Cosign > 0.500 ', 
my_tab/my_tab.ix["All"]

[ 0.63887758  0.64841615  0.58863789]
col_0  count  All
row_0            
False    112  112
True      41   41
All      153  153
Pilly Accuracy counting Cosign > 0.500 

col_0     count       All
row_0                    
False  0.732026  0.732026
True   0.267974  0.267974
All    1.000000  1.000000

In [21]:
print noPillyCoSign[0:3]
count_PAll = noPillyCoSign > 0.700 
#print count_PAll[0:5]
my_tab = pd.crosstab(index=count_PAll ,   columns="count",  margins=True  ) #  # Name the count column
print my_tab
print 'Not Pilly Accuracy counting Cosign > 0.500 ', 
my_tab/my_tab.ix["All"]

[ 0.70752386  0.69968089  0.49126875]
col_0  count  All
row_0            
False     55   55
True      22   22
All       77   77
Not Pilly Accuracy counting Cosign > 0.500 

col_0     count       All
row_0                    
False  0.714286  0.714286
True   0.285714  0.285714
All    1.000000  1.000000

#### Try Rocchio on ALL Yarn

In [18]:
def RocCoSAllYarn(testdf, protoV, uniqclasses):
    #Rocchio Text Categorization, prototype vectors for CoSine similarity required
    
    if len(protoV) <> len(uniqclasses):
        print 'Error!  # of Prototype Vectors do not match # of Unique Classes'
      
    testdf      = np.array(testdf)
    numTestrows = len(testdf)
    errorCount = 0.0
    resultlist = []
    cosinelist=[]

    for row in range(numTestrows):
    #for row in range(5):         #Testing with 10 only
        
        currCoS = np.float(-2.0)   #init with max value for each new row
        D = testdf[row]             #doc vector for categorization
                
        for iclass in range(len(protoV)):
            p = protoV[iclass]          #prototype vector to be compared to
            D_norm = np.linalg.norm(D)   #normalize test doc vector curr row
            p_norm = np.linalg.norm(p)
            cosine_sim = np.dot(D,p) / (D_norm * p_norm)
            #print 'curr cosine_sim for row ', row, cosine_sim
            if cosine_sim > currCoS:
                currCoS = cosine_sim
                classfound = uniqclasses[iclass]
        
        #store results
        resultlist.append( classfound ) 
        cosinelist.append(currCoS)
        
    return resultlist, np.array(cosinelist)

In [19]:
## call Rocchio function All Yarn
RocAllYarn , AllYarnCoSign = RocCoSAllYarn(yarn_all_np , pV, yclass_np)

In [20]:
AllYarnCoS_df = pd.DataFrame(AllYarnCoSign, columns=[ 'CoSign' ])
AllYarnCoS_df.describe(include="all")

CoSign
count  3657.000000
mean      0.465712
std       0.135104
min       0.041305
25%       0.376598
50%       0.480136
75%       0.564879
max       0.815875

In [32]:
AllYarnCoSign.shape

(3657L,)

In [21]:
print AllYarnCoSign[0:3]
#count_PAll = AllYarnCoSign > 0.5640 
count_PAll = AllYarnCoSign > 0.6000
#print count_PAll[0:5]
my_tab = pd.crosstab(index=count_PAll ,   columns="count",  margins=True  ) #  # Name the count column
print my_tab
print 'All Yarn Roccio Accuracy counting Cosign > 0.500 ', 
my_tab/my_tab.ix["All"]


[ 0.63887758  0.57687645  0.65455672]
col_0  count   All
row_0             
False   3077  3077
True     580   580
All     3657  3657
All Yarn Roccio Accuracy counting Cosign > 0.500 

col_0   count     All
row_0                
False  0.8414  0.8414
True   0.1586  0.1586
All    1.0000  1.0000

In [33]:
AllYarnCoS_df['yarn_id'] = yarn_all.index

In [35]:
AllYarnCoS_df.shape

(3657, 2)

In [40]:
import csv
import os
currentPath = os.getcwd()
print currentPath
%cd C:/Users/David/Google Drive/CSC478Proj/predict1

C:\Users\David\Google Drive\CSC478Proj\processing data2
C:\Users\David\Google Drive\CSC478Proj\predict1


In [42]:
AllYarnCoS_df.to_csv('Rocchio Cosign All Yarn All Features_June4.csv', index=False)